In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np

# random example images
images = np.random.randint(0, 255, (16, 128, 128, 3), dtype=np.uint8)

# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
# e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
# Define our sequence of augmentation steps that will be applied to every image
# All augmenters with per_channel=0.5 will sample one value _per image_
# in 50% of all cases. In all other cases they will sample new values
# _per channel_.
seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.2), # vertically flip 20% of all images
        # crop images by -5% to 10% of their height/width
        iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        ),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-90, 90), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 3),
            [
                sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
#                 iaa.OneOf([
#                     iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
#                     iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
#                     iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
#                 ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                # search either for all edges or for directed edges,
                # blend the result with the original image using a blobby mask
                iaa.SimplexNoiseAlpha(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0.5, 1.0)),
                    iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                ])),
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
#                 iaa.OneOf([
#                     iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
#                     iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
#                 ]),
#                 iaa.Invert(0.05, per_channel=True), # invert color channels
                iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                # either change the brightness of the whole image (sometimes
                # per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.5, 1.5), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-4, 0),
                        first=iaa.Multiply((0.5, 1.5), per_channel=True),
                        second=iaa.ContrastNormalization((0.5, 2.0))
                    )
                ]),
                iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                iaa.Grayscale(alpha=(0.0, 1.0)),
                sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
            random_order=True
        )
    ],
    random_order=True
)

print(images.shape)
images_aug = seq.augment_images(images)

(16, 128, 128, 3)


In [3]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def load_flower_data():
    # grab the list of images that we'll be describing
    print("[INFO] handling images...")
    TRAIN_ORIGINAL_DIR = './subsample/'
    TEST_DIR = './test/'

    # use this for full dataset
    train_images = [TRAIN_ORIGINAL_DIR + i for i in os.listdir(TRAIN_ORIGINAL_DIR)]
    test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
    train_images.sort(key=natural_keys)
    test_images.sort(key=natural_keys)

    # initialize the features matrix and labels list
    trainImage = []
    trainLabels = []
    testImage = []
    testLabels = []

    # loop over the input images
    for (i, imagePath) in enumerate(train_images):
        # extract the class label
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # read and resize image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        trainImage.append(img)
        trainLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
      # loop over the input images
    for (i, imagePath) in enumerate(test_images):
        # extract the class label
        # our images were named as labels.image_number.format
        # get the labels from the name of the images by extract the string before "."
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # extract CNN features in the image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        testImage.append(img)
        testLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


    trainImage = np.array(trainImage,dtype = float32)
    trainLabels = np.array(trainLabels)
    testImage = np.array(testImage,dtype = float32)
    testLabels = np.array(testLabels)
    print (trainImage.shape)
    
    trainImage = trainImage.astype(np.float32) / 255
    testImage = testImage.astype(np.float32) / 255
    
    le = preprocessing.LabelEncoder()
    le.fit(trainLabels)
    list(le.classes_)
    trainLabels = le.transform(trainLabels) 
    testLabels = le.transform(testLabels) 
    
    return trainImage, trainLabels, testImage, testLabels

In [4]:
trainImage, trainLabels, testImage, testLabels = load_flower_data()

nb_classes = 2

# Convert class vectors to binary class matrices.
trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
print (trainLabels)
testLabels = keras.utils.to_categorical(testLabels, nb_classes)
print (testLabels)
print (testLabels.shape)

np.save('../trainImage.npy', trainImage)
np.save('../trainLabels.npy', trainLabels)
np.save('../testImage.npy', testImage)
np.save('../testLabels.npy', testLabels)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage matrix: {:.2f}MB".format(
    (testImage.nbytes) / (1024 * 1000.0)))
print("[INFO] testLabels matrix: {:.4f}MB".format(
    (testLabels.nbytes) / (1024 * 1000.0)))


[INFO] handling images...
[INFO] processed 1000/8000
[INFO] processed 2000/8000
[INFO] processed 3000/8000
[INFO] processed 4000/8000
[INFO] processed 5000/8000
[INFO] processed 6000/8000
[INFO] processed 7000/8000
[INFO] processed 8000/8000
[INFO] processed 80/80
(8000, 128, 128, 3)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.

In [5]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]    
        # number of dense filters to use  
        self.nb_Dense = [128] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # train   
        self.global_step = tf.Variable(0, trainable=False)
        self.initial_lr = 0.001 #初始学习率
        self.decay_steps = 10
        self.decay_rate = 0.9
        self.lr = tf.train.exponential_decay( self.initial_lr,
                                     global_step=self.global_step,
                                     decay_steps=self.decay_steps,
                                     decay_rate=self.decay_rate)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        # name for checkpoint
        self.model_name = 'CNN_C%d_D%d_Kernel(%d,%d)_%d_lrdecay0.0001' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))    
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y= np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y= np.load('../testLabels.npy')
        
        print ("Training:",self.train_x.shape[0])
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 

            #输入,卷积核为3*3 输出维度为32
            net = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1'               # 命名用于获取变量
                                    )
            print("CNN:",net.get_shape())

            if self.batch_normalization_control:
                net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            net = tf.layers.max_pooling2d(inputs = net,
                                          pool_size = self.pool_size,
                                          strides = (2, 2),
                                          padding='same',
                                          name='pool_conv_' + str(1)
                                         )
            print("CNN:",net.get_shape())
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )

                print("CNN:",net.get_shape())

            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
            
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'_train_hist.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
#                 batch_x = np.array(seq.augment_images(np.array(batch_x,dtype = uint8)),dtype = float32)
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status            
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, 
                                                                                    self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                rand_index = np.random.choice(len(self.train_x), size=batch_size)
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                   self.y: batch_y_tes,
                                                                   self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
            
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
                
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append(test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '_train_hist.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '_train_hist.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
    
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")
        
    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')
                          
        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [ ]:
gan_type = 'CNN'
dataset = '4_Flowers'
epoch = 100
batch_size = 40
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
    #CNN.test(epoch)
        
sess.close()
        

# lrdecay
# Avg per epoch ptime: 12.63, total 100 epochs ptime: 1299.20
#  [*] Training finished!
#  [*] Best Epoch:  26 , Accuracy:  1.0
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001-26
#  [*] Finished testing Best Epoch: 26 , accuracy:  1.0 !

Training: 8000
CNN:x (40, 128, 128, 3)
CNN: (40, 128, 128, 32)
CNN: (40, 64, 64, 32)
CNN: (40, 64, 64, 64)
CNN: (40, 32, 32, 64)
CNN: (40, 32, 32, 64)
CNN: (40, 16, 16, 64)
CNN: (40, 16, 16, 64)
CNN: (40, 8, 8, 64)
CNN: (40, 8, 8, 128)
CNN: (40, 4, 4, 128)
CNN: (40, 2048)
CNN:out_logit (40, 2)
------------------------
CNN:x (40, 128, 128, 3)
CNN: (40, 128, 128, 32)
CNN: (40, 64, 64, 32)
CNN: (40, 64, 64, 64)
CNN: (40, 32, 32, 64)
CNN: (40, 32, 32, 64)
CNN: (40, 16, 16, 64)
CNN: (40, 16, 16, 64)
CNN: (40, 8, 8, 64)
CNN: (40, 8, 8, 128)
CNN: (40, 4, 4, 128)
CNN: (40, 2048)
CNN:out_logit (40, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/beta:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/gamma:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_2/kernel:0 (float32_ref 3x3x32x64) [18432, bytes: 73728]
cnn/conv

Epoch: [ 1] [ 108/ 200] time: 8.1783, loss: 0.17324993
Epoch: [ 1] [ 109/ 200] time: 8.2418, loss: 0.34772837
Epoch: [ 1] [ 110/ 200] time: 8.3069, loss: 0.30336976
Epoch: [ 1] [ 111/ 200] time: 8.3707, loss: 0.17711470
Epoch: [ 1] [ 112/ 200] time: 8.4345, loss: 0.33191928
Epoch: [ 1] [ 113/ 200] time: 8.4985, loss: 0.17767999
Epoch: [ 1] [ 114/ 200] time: 8.5628, loss: 0.34859136
Epoch: [ 1] [ 115/ 200] time: 8.6267, loss: 0.51258492
Epoch: [ 1] [ 116/ 200] time: 8.6909, loss: 0.48148239
Epoch: [ 1] [ 117/ 200] time: 8.7556, loss: 0.55332381
Epoch: [ 1] [ 118/ 200] time: 8.8190, loss: 0.34069276
Epoch: [ 1] [ 119/ 200] time: 8.8834, loss: 0.26489913
Epoch: [ 1] [ 120/ 200] time: 8.9484, loss: 0.32280898
Epoch: [ 1] [ 121/ 200] time: 9.0139, loss: 0.28750747
Epoch: [ 1] [ 122/ 200] time: 9.0777, loss: 0.39731240
Epoch: [ 1] [ 123/ 200] time: 9.1424, loss: 0.26786232
Epoch: [ 1] [ 124/ 200] time: 9.2057, loss: 0.34339237
Epoch: [ 1] [ 125/ 200] time: 9.2681, loss: 0.35190010
Epoch: [ 1

Epoch: [ 2] [  56/ 200] time: 18.3794, loss: 0.20103449
Epoch: [ 2] [  57/ 200] time: 18.4427, loss: 0.15636821
Epoch: [ 2] [  58/ 200] time: 18.5035, loss: 0.13940686
Epoch: [ 2] [  59/ 200] time: 18.5628, loss: 0.16045368
Epoch: [ 2] [  60/ 200] time: 18.6241, loss: 0.19717801
Epoch: [ 2] [  61/ 200] time: 18.6850, loss: 0.21202996
Epoch: [ 2] [  62/ 200] time: 18.7457, loss: 0.32870921
Epoch: [ 2] [  63/ 200] time: 18.8058, loss: 0.21366212
Epoch: [ 2] [  64/ 200] time: 18.8686, loss: 0.14701639
Epoch: [ 2] [  65/ 200] time: 18.9300, loss: 0.31528491
Epoch: [ 2] [  66/ 200] time: 18.9888, loss: 0.35946882
Epoch: [ 2] [  67/ 200] time: 19.0474, loss: 0.22292738
Epoch: [ 2] [  68/ 200] time: 19.1086, loss: 0.28321567
Epoch: [ 2] [  69/ 200] time: 19.1698, loss: 0.21101335
Epoch: [ 2] [  70/ 200] time: 19.2295, loss: 0.25307912
Epoch: [ 2] [  71/ 200] time: 19.2890, loss: 0.10293701
Epoch: [ 2] [  72/ 200] time: 19.3503, loss: 0.18100509
Epoch: [ 2] [  73/ 200] time: 19.4109, loss: 0.1

Epoch: [ 3] [   4/ 200] time: 28.3068, loss: 0.14914805
Epoch: [ 3] [   5/ 200] time: 28.3664, loss: 0.05722075
Epoch: [ 3] [   6/ 200] time: 28.4249, loss: 0.17528751
Epoch: [ 3] [   7/ 200] time: 28.4838, loss: 0.33353814
Epoch: [ 3] [   8/ 200] time: 28.5451, loss: 0.18177809
Epoch: [ 3] [   9/ 200] time: 28.6053, loss: 0.07815307
Epoch: [ 3] [  10/ 200] time: 28.6640, loss: 0.09735275
Epoch: [ 3] [  11/ 200] time: 28.7226, loss: 0.09207193
Epoch: [ 3] [  12/ 200] time: 28.7839, loss: 0.26215422
Epoch: [ 3] [  13/ 200] time: 28.8448, loss: 0.11865458
Epoch: [ 3] [  14/ 200] time: 28.9035, loss: 0.10789844
Epoch: [ 3] [  15/ 200] time: 28.9632, loss: 0.13440420
Epoch: [ 3] [  16/ 200] time: 29.0245, loss: 0.26532561
Epoch: [ 3] [  17/ 200] time: 29.0846, loss: 0.18225878
Epoch: [ 3] [  18/ 200] time: 29.1432, loss: 0.18933123
Epoch: [ 3] [  19/ 200] time: 29.2018, loss: 0.07680479
Epoch: [ 3] [  20/ 200] time: 29.2632, loss: 0.19124210
Epoch: [ 3] [  21/ 200] time: 29.3237, loss: 0.1

Epoch: [ 3] [ 152/ 200] time: 37.1686, loss: 0.12224440
Epoch: [ 3] [ 153/ 200] time: 37.2292, loss: 0.11968960
Epoch: [ 3] [ 154/ 200] time: 37.2880, loss: 0.09686720
Epoch: [ 3] [ 155/ 200] time: 37.3466, loss: 0.25007576
Epoch: [ 3] [ 156/ 200] time: 37.4078, loss: 0.07894477
Epoch: [ 3] [ 157/ 200] time: 37.4680, loss: 0.03117655
Epoch: [ 3] [ 158/ 200] time: 37.5269, loss: 0.23448543
Epoch: [ 3] [ 159/ 200] time: 37.5868, loss: 0.11349100
Epoch: [ 3] [ 160/ 200] time: 37.6479, loss: 0.15924773
Epoch: [ 3] [ 161/ 200] time: 37.7086, loss: 0.11664064
Epoch: [ 3] [ 162/ 200] time: 37.7671, loss: 0.18038416
Epoch: [ 3] [ 163/ 200] time: 37.8257, loss: 0.21016403
Epoch: [ 3] [ 164/ 200] time: 37.8868, loss: 0.11095230
Epoch: [ 3] [ 165/ 200] time: 37.9484, loss: 0.11466742
Epoch: [ 3] [ 166/ 200] time: 38.0071, loss: 0.38509652
Epoch: [ 3] [ 167/ 200] time: 38.0657, loss: 0.25822663
Epoch: [ 3] [ 168/ 200] time: 38.1270, loss: 0.03939252
Epoch: [ 3] [ 169/ 200] time: 38.1876, loss: 0.1

...

Epoch: [97] [ 100/ 200] time: 1253.9422, loss: 0.00000646
Epoch: [97] [ 101/ 200] time: 1254.0024, loss: 0.00001618
Epoch: [97] [ 102/ 200] time: 1254.0617, loss: 0.00000014
Epoch: [97] [ 103/ 200] time: 1254.1205, loss: 0.00000061
Epoch: [97] [ 104/ 200] time: 1254.1819, loss: 0.00000000
Epoch: [97] [ 105/ 200] time: 1254.2425, loss: 0.00000012
Epoch: [97] [ 106/ 200] time: 1254.3014, loss: 0.00000330
Epoch: [97] [ 107/ 200] time: 1254.3603, loss: 0.00006452
Epoch: [97] [ 108/ 200] time: 1254.4216, loss: 0.00015192
Epoch: [97] [ 109/ 200] time: 1254.4821, loss: 0.00001498
Epoch: [97] [ 110/ 200] time: 1254.5410, loss: 0.00000289
Epoch: [97] [ 111/ 200] time: 1254.5999, loss: 0.00002926
Epoch: [97] [ 112/ 200] time: 1254.6615, loss: 0.00003385
Epoch: [97] [ 113/ 200] time: 1254.7221, loss: 0.00003973
Epoch: [97] [ 114/ 200] time: 1254.7809, loss: 0.00000008
Epoch: [97] [ 115/ 200] time: 1254.8398, loss: 0.00014223
Epoch: [97] [ 116/ 200] time: 1254.9012, loss: 0.00007357
Epoch: [97] [ 

Epoch: [98] [  44/ 200] time: 1263.5423, loss: 0.00000004
Epoch: [98] [  45/ 200] time: 1263.6033, loss: 0.00003943
Epoch: [98] [  46/ 200] time: 1263.6632, loss: 0.00000196
Epoch: [98] [  47/ 200] time: 1263.7224, loss: 0.00000100
Epoch: [98] [  48/ 200] time: 1263.7837, loss: 0.00289801
Epoch: [98] [  49/ 200] time: 1263.8449, loss: 0.00000152
Epoch: [98] [  50/ 200] time: 1263.9040, loss: 0.00000001
Epoch: [98] [  51/ 200] time: 1263.9654, loss: 0.00050711
Epoch: [98] [  52/ 200] time: 1264.0269, loss: 0.00000789
Epoch: [98] [  53/ 200] time: 1264.0878, loss: 0.00000590
Epoch: [98] [  54/ 200] time: 1264.1467, loss: 0.00000049
Epoch: [98] [  55/ 200] time: 1264.2056, loss: 0.00000001
Epoch: [98] [  56/ 200] time: 1264.2671, loss: 0.00000002
Epoch: [98] [  57/ 200] time: 1264.3280, loss: 0.00001691
Epoch: [98] [  58/ 200] time: 1264.3869, loss: 0.00000024
Epoch: [98] [  59/ 200] time: 1264.4458, loss: 0.00025736
Epoch: [98] [  60/ 200] time: 1264.5071, loss: 0.00000026
Epoch: [98] [ 

Epoch: [98] [ 188/ 200] time: 1272.1981, loss: 0.00000060
Epoch: [98] [ 189/ 200] time: 1272.2586, loss: 0.00000009
Epoch: [98] [ 190/ 200] time: 1272.3173, loss: 0.00000003
Epoch: [98] [ 191/ 200] time: 1272.3763, loss: 0.00006616
Epoch: [98] [ 192/ 200] time: 1272.4377, loss: 0.00000128
Epoch: [98] [ 193/ 200] time: 1272.4989, loss: 0.00001511
Epoch: [98] [ 194/ 200] time: 1272.5582, loss: 0.00000018
Epoch: [98] [ 195/ 200] time: 1272.6172, loss: 0.00000008
Epoch: [98] [ 196/ 200] time: 1272.6787, loss: 0.00000006
Epoch: [98] [ 197/ 200] time: 1272.7400, loss: 0.00000004
Epoch: [98] [ 198/ 200] time: 1272.7987, loss: 0.00486554
Epoch: [98] [ 199/ 200] time: 1272.8582, loss: 0.00009658
[98/100] - ptime: 12.6083 loss: 0.00013178 acc: 1.00000 lr: 0.00035610
Epoch: [99] [   0/ 200] time: 1273.9024, loss: 0.00000034
Epoch: [99] [   1/ 200] time: 1273.9622, loss: 0.00000002
Epoch: [99] [   2/ 200] time: 1274.0211, loss: 0.00000033
Epoch: [99] [   3/ 200] time: 1274.0799, loss: 0.00000704
E

Epoch: [99] [ 132/ 200] time: 1281.8360, loss: 0.00000004
Epoch: [99] [ 133/ 200] time: 1281.8970, loss: 0.00000441
Epoch: [99] [ 134/ 200] time: 1281.9570, loss: 0.00000655
Epoch: [99] [ 135/ 200] time: 1282.0158, loss: 0.00000001
Epoch: [99] [ 136/ 200] time: 1282.0771, loss: 0.00000133
Epoch: [99] [ 137/ 200] time: 1282.1383, loss: 0.00019362
Epoch: [99] [ 138/ 200] time: 1282.1971, loss: 0.00000006
Epoch: [99] [ 139/ 200] time: 1282.2559, loss: 0.00001347
Epoch: [99] [ 140/ 200] time: 1282.3171, loss: 0.00002048
Epoch: [99] [ 141/ 200] time: 1282.3782, loss: 0.00000000
Epoch: [99] [ 142/ 200] time: 1282.4372, loss: 0.00030501
Epoch: [99] [ 143/ 200] time: 1282.4966, loss: 0.00001817
Epoch: [99] [ 144/ 200] time: 1282.5580, loss: 0.11435093
Epoch: [99] [ 145/ 200] time: 1282.6188, loss: 0.00000002
Epoch: [99] [ 146/ 200] time: 1282.6782, loss: 0.00000661
Epoch: [99] [ 147/ 200] time: 1282.7382, loss: 0.01035286
Epoch: [99] [ 148/ 200] time: 1282.7995, loss: 0.00000369
Epoch: [99] [ 

Epoch: [100] [  72/ 200] time: 1291.1813, loss: 0.00000002
Epoch: [100] [  73/ 200] time: 1291.2411, loss: 0.00000023
Epoch: [100] [  74/ 200] time: 1291.3002, loss: 0.00000603
Epoch: [100] [  75/ 200] time: 1291.3597, loss: 0.00000000
Epoch: [100] [  76/ 200] time: 1291.4209, loss: 0.00000628
Epoch: [100] [  77/ 200] time: 1291.4817, loss: 0.00000925
Epoch: [100] [  78/ 200] time: 1291.5408, loss: 0.00001647
Epoch: [100] [  79/ 200] time: 1291.5997, loss: 0.00000043
Epoch: [100] [  80/ 200] time: 1291.6610, loss: 0.00293014
Epoch: [100] [  81/ 200] time: 1291.7215, loss: 0.04083811
Epoch: [100] [  82/ 200] time: 1291.7804, loss: 0.00000750
Epoch: [100] [  83/ 200] time: 1291.8393, loss: 0.00000231
Epoch: [100] [  84/ 200] time: 1291.9005, loss: 0.00000018
Epoch: [100] [  85/ 200] time: 1291.9618, loss: 0.00000007
Epoch: [100] [  86/ 200] time: 1292.0209, loss: 0.00013780
Epoch: [100] [  87/ 200] time: 1292.0804, loss: 0.00000419
Epoch: [100] [  88/ 200] time: 1292.1416, loss: 0.000522

In [ ]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放